# MongoDB basic test functions
## Function list : 
        test_insert_bulk, 
        test_insert_one, 
        find_some_data, 
        find_all_data, 
        update_some_data, 
        update_all_data

### Dependence : service kafka, kafka zookeeper, service mongoDB, jaeger-all-in-one, utils.py, mongodb_test.py
### Parameter list : 
        choose test function (required) : --function,
        pass query string : --query,
        pass new value for update operation : --value,
        choose topic : --topic,
        choose mongodb collection : --collection,
        pass name of jaeger tarcer : --tracer

## Insertion test :
### Insert data from a topic line by line. Optional parameters : --topic, --collection, --tracer

In [ ]:
%run ../TimeSeriesTools/mongodb_test.py --function test_insert_one

### Insert data from a topic in bulk. Optional parameters : --topic, --collection, --tracer

In [ ]:
%run ../TimeSeriesTools/mongodb_test.py --function test_insert_bulk

## Read test : 
### Read all data from a mongodb collection. Optional parameters : --collection, --tracer

In [ ]:
%run ../TimeSeriesTools/mongodb_test.py --function find_all_data

### Select some data from a mongodb collection by a query. Optional parameters : --collection, --query, --tracer

In [ ]:
%run ../TimeSeriesTools/mongodb_test.py --function find_some_data --query '{"Heure":{"$regex":"11:"}}'

## Update test :
### Update all documents in a mongodb collection. Optional parameters : --value, --collection, --tracer

In [ ]:
%run ../TimeSeriesTools/mongodb_test.py --function update_all_data --value '{"$set":{"C2H6":"0"}}'

### Update selected documents in a mongodb collection. Optional parameters : --collection, --query, --value, --tracer

In [ ]:
%run ../TimeSeriesTools/mongodb_test.py --function update_some_data --query '{"Heure":{"$regex":"02/10"}}' --value '{"$set":{"C2H6":"1"}}'